# Sayou Document Quick Start

**Sayou Fabric**의 문서 파싱 게이트웨이인 `sayou-document`를 시연합니다.
이 라이브러리는 PDF, DOCX, PPTX, XLSX 뿐만 아니라 이미지 파일까지 하나의 통합된 **Document Object Model (DOM)**로 변환합니다.

**핵심 시나리오:**
1. **Smart Routing:** 확장자에 따라 자동으로 적절한 파서(Parser)를 선택합니다.
2. **Image Processing:** 이미지를 자동으로 PDF로 변환하고 OCR을 수행하여 텍스트를 추출합니다.
3. **Structured Output:** 모든 문서를 통일된 JSON 구조로 변환하여 후처리(Refinery/Chunking)를 돕습니다.

In [ ]:
import os
import logging
from sayou.document.pipeline import DocumentPipeline

# 로그 레벨 설정 (파싱 과정을 자세히 보기 위해 INFO 설정)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

print("Import Successful! Ready to parse.")

## 1. 파이프라인 초기화 (Initialization)

`DocumentPipeline`을 초기화합니다.
`use_default_ocr=True` 옵션을 주면, 시스템에 설치된 Tesseract 엔진을 자동으로 찾아 로드합니다. (Tesseract가 없으면 경고 로그만 뜨고 OCR 기능은 꺼집니다.)

In [ ]:
# 파이프라인 생성 (OCR 자동 활성화 시도)
pipeline = DocumentPipeline(use_default_ocr=True)
pipeline.initialize()

## 2. 시나리오: 이미지 자동 변환 및 OCR

가장 흥미로운 기능인 **"이미지 -> 문서 변환"**을 테스트해 봅니다.
테스트를 위해 Python 코드로 텍스트가 포함된 이미지를 즉석에서 생성하겠습니다.

In [ ]:
# [Helper] 테스트용 텍스트 이미지 생성 (Pillow 필요)
IMG_PATH = "test_ocr_sample.png"

try:
    from PIL import Image, ImageDraw
    
    # 흰 배경 이미지 생성
    img = Image.new('RGB', (500, 200), color=(255, 255, 255))
    d = ImageDraw.Draw(img)
    
    # 텍스트 쓰기 (기본 폰트 사용)
    d.text((20, 50), "Sayou Document Pipeline", fill=(0, 0, 0))
    d.text((20, 80), "Automatic Image-to-PDF Conversion", fill=(0, 0, 128))
    d.text((20, 110), "OCR Test Successful!", fill=(0, 128, 0))
    
    img.save(IMG_PATH)
    print(f"Created test image: {IMG_PATH}")
    
except ImportError:
    print("Pillow not installed. Skipping image creation.")

## 3. 파이프라인 실행 (Run)

이미지 파일의 바이너리를 읽어서 파이프라인에 던집니다.
파이프라인은 내부적으로 **[Image -> PDF 변환 -> PDF Parser -> OCR 수행]** 과정을 자동으로 처리합니다.

In [ ]:
if os.path.exists(IMG_PATH):
    # 1. 파일 읽기
    with open(IMG_PATH, "rb") as f:
        file_bytes = f.read()

    print(f"Processing '{IMG_PATH}' ({len(file_bytes)} bytes)...")

    try:
        # 2. 파이프라인 실행
        doc = pipeline.run(file_bytes, IMG_PATH)

        if doc:
            print("\n✅ Parsing Complete!")
            print(f"File Name: {doc.file_name}")
            print(f"Doc Type : {doc.doc_type} (Converted internally)")
            print(f"Page Count: {doc.page_count}")
            
            # 3. 결과 확인 (첫 번째 페이지의 요소들)
            first_page = doc.pages[0]
            print(f"\n--- Extracted Content (Page 1) ---")
            
            for elem in first_page.elements:
                # elem.text 프로퍼티가 OCR 결과를 반환합니다.
                print(f"[{elem.type}] {elem.text}")
                
        else:
            print("❌ Parsing returned None.")

    except Exception as e:
        print(f"❌ Error during processing: {e}")
else:
    print("Test image not found.")

## 4. 구조화된 데이터 확인 (JSON Export)

파싱된 결과(`Document` 객체)는 Pydantic 모델이므로, `model_dump_json()`을 통해 손쉽게 JSON으로 직렬화할 수 있습니다. 이 데이터는 다음 단계인 **Refinery**로 전달되기에 최적화되어 있습니다.

In [ ]:
if doc:
    # JSON 문자열로 변환
    json_output = doc.model_dump_json(indent=2)
    
    # 앞부분만 출력하여 구조 확인
    print(json_output[:500] + "\n... (truncated)")
    
    # 파일로 저장 (선택)
    with open("result_doc.json", "w", encoding="utf-8") as f:
        f.write(json_output)
    print("\nSaved full result to 'result_doc.json'")

In [ ]:
# [Cleanup] 테스트 파일 정리
if os.path.exists(IMG_PATH):
    os.remove(IMG_PATH)
if os.path.exists("result_doc.json"):
    os.remove("result_doc.json")
    
print("Cleanup complete.")